In [29]:
import pandas as pd
import json

#### טעינת כל הטבלאות

In [ ]:
raw_df      = pd.read_csv("raw_plan_data.csv",      encoding="utf-8")
finished_df = pd.read_csv("finished_projects.csv",  encoding="utf-8")
features_df = pd.read_csv("all_finish_features.csv",encoding="utf-8")

#### איחוד שמות עמודות ב-raw_df וב-finished_df


In [ ]:
raw_df = raw_df.rename(columns={
    "pr":           "מספר פרויקט",
    "project_name": "שם הפרויקט",
    "thum":         "תחום",
    "prof_name":    "סוג התוכנית"
})
if "pr" in finished_df.columns:
    finished_df = finished_df.rename(columns={"pr":"מספר פרויקט"})
if "pr" in features_df.columns:
    features_df = features_df.rename(columns={"pr":"מספר פרויקט"})

#### **check**

In [33]:
raw_df.head()

,מספר פרויקט,שם הפרויקט,תחום,סוג התוכנית,prog_name,gilayon,miflas,num,girsa,tareidkun
0,10,ניסיון,מגורים,אדריכלות,sfsdfsdf,5,2,3,8,18/04/2023 00:00
1,10,ניסיון,מגורים,אדריכלות,תכנית בנייה,102,2,3,8,18/04/2023 00:00
2,10,ניסיון,מגורים,בטיחות,222,1234,2,6,8,18/04/2023 00:00
3,10,ניסיון,מגורים,בטיחות,פרוגרמת בטיחות,0,1,6,8,18/04/2023 00:00
4,10,ניסיון,מגורים,קונסטרוקציה,חפירה ודיפון,00_20,3,8,8,18/04/2023 00:00


#### וידוא טיפוס int על 'מספר פרויקט'

In [34]:
for df in (raw_df, finished_df, features_df):
    df["מספר פרויקט"] = pd.to_numeric(df["מספר פרויקט"], errors="coerce")
    df.dropna(subset=["מספר פרויקט"], inplace=True)
    df["מספר פרויקט"] = df["מספר פרויקט"].astype(int)

#### סינון raw_df לפי שני תנאים (OR)

In [35]:
mask_finished = raw_df["מספר פרויקט"].isin(finished_df["מספר פרויקט"])

valid_projects = features_df.loc[
    (features_df["שלב הפרויקט 2"].isin(["בדק", "הסתיים"])) |
    (features_df["שלב משני 2"].isin(["גמרים", "טופס 4", "מסירות", "שלד עילי"])) |
    (features_df["תאריך קבלת טופס 4"].notna()),
    "מספר פרויקט"
].unique()
mask_features = raw_df["מספר פרויקט"].isin(valid_projects)

filtered_df = raw_df[mask_finished | mask_features].copy()

#### **check**

In [36]:
filtered_df.head()

,מספר פרויקט,שם הפרויקט,תחום,סוג התוכנית,prog_name,gilayon,miflas,num,girsa,tareidkun
42,185,כיכר המדינה,מגורים,חשמל,תכנית פיתוח,14169-AR1(HH),5,6,1,19/07/2022 00:00
43,185,כיכר המדינה,מגורים,מודד,מפה טופוגרפית - כללי,0,4,5,1,19/07/2022 00:00
44,185,כיכר המדינה,מגורים,קונסטרוקציה,מגדל C תכנית זיון תקרת מרתף 3- במפלס 8.65+,C-521,7,3,1,19/07/2022 00:00
45,185,כיכר המדינה,מגורים,קונסטרוקציה,מגדל C תכנית תקרת מרתף 3- במפלס 8.65+,C-520,7,3,1,19/07/2022 00:00
46,185,כיכר המדינה,מגורים,קונסטרוקציה,תכנית חתכי קירות דיפון (המשך),003B - ק,4,3,1,19/07/2022 00:00


#### Pivot וספירת תכניות לכל פרויקט וסוג תוכנית


In [ ]:
pivot_df = (
    filtered_df
    .groupby(["מספר פרויקט", "סוג התוכנית"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

#### **check**

In [39]:
pivot_df.head()

סוג התוכנית,מספר פרויקט,אבן,אגרונום,אדריכלות,אדריכלות נוף,אדריכלות פנים,איטום,איכות הסביבה,אינסטלציה,איתור ומיפוי תשתיות תת קרקעיות,...,רישוי,שילוט וגידור,תאום מערכות,תאורה,תברואה,תיאום הנדסי,תכן מבנה,תנועה,תקשורת,תשתיות רטובות
0,185,0,2,2,0,0,0,0,0,6,...,0,0,0,0,0,3,0,0,0,0
1,762,0,2,253,27,0,16,0,0,0,...,19,2,75,5,155,0,0,11,2,0
2,765,0,0,179,9,141,7,0,0,0,...,0,0,17,2,144,0,0,8,3,0
3,775,0,0,269,20,116,1,0,151,0,...,0,0,0,7,0,45,0,8,0,0
4,798,0,0,136,1,0,24,0,0,0,...,0,0,92,0,70,11,0,2,0,0


In [40]:
print("מספר הפרויקטים שקיימת עבורם רשימת תכניות:", len(pivot_df))

מספר הפרויקטים שקיימת עבורם רשימת תכניות: 66


#### טבלת פרויקטים עם שם ותחום

In [ ]:
names_df = (
    features_df[["מספר פרויקט", "שם הפרויקט"]]
    .drop_duplicates()
)

types_df = (
    filtered_df[["מספר פרויקט", "תחום"]]
    .drop_duplicates()
)

project_info = (
    names_df
    .merge(types_df, on="מספר פרויקט", how="inner")
    .rename(columns={"תחום":"תחום"})
)

#### **check** <br>

#### 1
names_df &rarr; features_df &rarr; all_finish_features

In [42]:
names_df.head()

,מספר פרויקט,שם הפרויקט
0,1001,עץ החיים
1,1002,מגדלי גינדי תל אביב - מגדל 4
2,1004,עד 120 נס ציונה
3,1005,הרחבת קניון גזית כפר סבא
4,1006,מרכז לוגיסטי הסדנה חולון


#### 2

types_df &rarr; filtered_df

In [43]:
types_df.head()

,מספר פרויקט,תחום
42,185,מגורים
158,762,מסחר
1679,765,מגורים
2940,775,מגורים
4296,798,משרדים


#### 3 
*Merge* &rarr; types_df <> names_df

In [44]:
project_info.head()

,מספר פרויקט,שם הפרויקט,תחום
0,1002,מגדלי גינדי תל אביב - מגדל 4,מגורים
1,1006,מרכז לוגיסטי הסדנה חולון,תעשיה
2,1016,בית מנצור 2,משרדים
3,1017,חברת חשמל- מתחם גן שורק,משרדים
4,1019,זרחין רעננה,משרדים


#### מיזוג לקבלת הטבלה הסופית

In [ ]:
final_df = project_info.merge(pivot_df, on="מספר פרויקט", how="left")

In [47]:
final_df.head()

,מספר פרויקט,שם הפרויקט,תחום,אבן,אגרונום,אדריכלות,אדריכלות נוף,אדריכלות פנים,איטום,איכות הסביבה,...,רישוי,שילוט וגידור,תאום מערכות,תאורה,תברואה,תיאום הנדסי,תכן מבנה,תנועה,תקשורת,תשתיות רטובות
0,1002,מגדלי גינדי תל אביב - מגדל 4,מגורים,0,0,159,8,19,2,0,...,0,0,42,4,58,2,0,6,0,0
1,1006,מרכז לוגיסטי הסדנה חולון,תעשיה,0,0,46,3,0,2,0,...,0,0,0,0,0,16,0,1,0,0
2,1016,בית מנצור 2,משרדים,0,0,39,8,11,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1017,חברת חשמל- מתחם גן שורק,משרדים,0,0,456,39,0,13,0,...,0,30,64,0,0,0,0,12,0,0
4,1019,זרחין רעננה,משרדים,0,0,95,17,32,1,0,...,0,0,0,18,0,34,0,6,0,0


In [52]:
final_df.head()

,מספר פרויקט,שם הפרויקט,תחום,אבן,אגרונום,אדריכלות,אדריכלות נוף,אדריכלות פנים,איטום,איכות הסביבה,...,רישוי,שילוט וגידור,תאום מערכות,תאורה,תברואה,תיאום הנדסי,תכן מבנה,תנועה,תקשורת,תשתיות רטובות
0,1002,מגדלי גינדי תל אביב - מגדל 4,מגורים,0,0,159,8,19,2,0,...,0,0,42,4,58,2,0,6,0,0
1,1006,מרכז לוגיסטי הסדנה חולון,תעשיה,0,0,46,3,0,2,0,...,0,0,0,0,0,16,0,1,0,0
2,1016,בית מנצור 2,משרדים,0,0,39,8,11,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1017,חברת חשמל- מתחם גן שורק,משרדים,0,0,456,39,0,13,0,...,0,30,64,0,0,0,0,12,0,0
4,1019,זרחין רעננה,משרדים,0,0,95,17,32,1,0,...,0,0,0,18,0,34,0,6,0,0


#### הסרת כפילויות עמודות אם יש

In [53]:
final_df = final_df.loc[:, ~final_df.columns.duplicated()]

In [54]:
final_df.head()

,מספר פרויקט,שם הפרויקט,תחום,אבן,אגרונום,אדריכלות,אדריכלות נוף,אדריכלות פנים,איטום,איכות הסביבה,...,רישוי,שילוט וגידור,תאום מערכות,תאורה,תברואה,תיאום הנדסי,תכן מבנה,תנועה,תקשורת,תשתיות רטובות
0,1002,מגדלי גינדי תל אביב - מגדל 4,מגורים,0,0,159,8,19,2,0,...,0,0,42,4,58,2,0,6,0,0
1,1006,מרכז לוגיסטי הסדנה חולון,תעשיה,0,0,46,3,0,2,0,...,0,0,0,0,0,16,0,1,0,0
2,1016,בית מנצור 2,משרדים,0,0,39,8,11,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1017,חברת חשמל- מתחם גן שורק,משרדים,0,0,456,39,0,13,0,...,0,30,64,0,0,0,0,12,0,0
4,1019,זרחין רעננה,משרדים,0,0,95,17,32,1,0,...,0,0,0,18,0,34,0,6,0,0


#### סדר עמודות: שם הפרויקט, מספר פרויקט, תחום, ואחר-כך כל שדות התכניות

In [55]:
plan_cols = [c for c in final_df.columns if c not in ["שם הפרויקט", "מספר פרויקט", "תחום"]]
final_df = final_df[["שם הפרויקט", "מספר פרויקט", "תחום"] + plan_cols]

#### שמירה של הטבלה הסופית

In [56]:
final_df.to_csv("final_project_summary.csv", index=False, encoding="utf-8-sig")

#### חישוב היחסים לכל סוג תוכנית בהתבסס על אדריכלות

In [57]:
if "אדריכלות" not in final_df.columns:
    raise KeyError("עמודת 'אדריכלות' לא נמצאה בטבלה הסופית")

base = final_df["אדריכלות"].values
ratios_df = final_df.copy()
for col in plan_cols:
    ratios_df[f"ratio_{col}"] = ratios_df[col].values / base

C:\Users\noams\AppData\Local\Temp\ipykernel_24072\227765078.py:7: RuntimeWarning: invalid value encountered in divide
  ratios_df[f"ratio_{col}"] = ratios_df[col].values / base
C:\Users\noams\AppData\Local\Temp\ipykernel_24072\227765078.py:7: RuntimeWarning: divide by zero encountered in divide
  ratios_df[f"ratio_{col}"] = ratios_df[col].values / base
C:\Users\noams\AppData\Local\Temp\ipykernel_24072\227765078.py:7: RuntimeWarning: invalid value encountered in divide
  ratios_df[f"ratio_{col}"] = ratios_df[col].values / base
C:\Users\noams\AppData\Local\Temp\ipykernel_24072\227765078.py:7: RuntimeWarning: invalid value encountered in divide
  ratios_df[f"ratio_{col}"] = ratios_df[col].values / base
C:\Users\noams\AppData\Local\Temp\ipykernel_24072\227765078.py:7: RuntimeWarning: divide by zero encountered in divide
  ratios_df[f"ratio_{col}"] = ratios_df[col].values / base
C:\Users\noams\AppData\Local\Temp\ipykernel_24072\227765078.py:7: RuntimeWarning: invalid value encountered in di

#### חישוב ממוצע היחסים לפי תחום 

In [58]:
ratio_cols = [c for c in ratios_df.columns if c.startswith("ratio_")]
mean_ratios_df = ratios_df.groupby("תחום")[ratio_cols].mean().reset_index()

### שמירה כ-JSON

*הקובץ mean_ratios.json מכיל, לכל סוג פרויקט (כגון מגורים, משרדים וכד’), את היחס הממוצע בין מספר תכניות X למספר תכניות אדריכלות שחישבנו בעבר. כך שבכל פעם שנרצה לנבא עבור פרויקט חדש כמה תכניות נדרשות מכל סוג, במקום להריץ שוב את כל תהליכי ה-pivot וה-groupby, פשוט טוענים את ה-JSON, שולפים משם את היחס הרלוונטי ומכפילים אותו במספר תכניות האדריכלות שהוזן. זה חוסך זמן, עומס חישובי ומאפשר לאפליקציה (API או Streamlit) להחזיר תוצאה מיידית.*

In [59]:
mean_ratios = mean_ratios_df.set_index("תחום").to_dict(orient="index")
with open("mean_ratios.json", "w", encoding="utf-8") as f:
    json.dump(mean_ratios, f, ensure_ascii=False, indent=2)